# Creating flows in jobflow

In this tutorial, you will:

- Learn about the `Flow` object.
- Set the configuration settings of a flow and its jobs.

The purpose of this tutorial is to delve into the basic functionality of flows and gain a feeling for what is possible. Later tutorials will describe how to create dynamic flows.

## Creating job objects

The building block of jobflows are `Job` objects. Jobs are delayed calls to python functions whose outputs are stored in a database. The easiest way to create a job is using the `@job` decorator. The job decorator can be applied to any function, even those with optional parameters.


In [35]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

We will start by defining two simple jobs that we will stitch together into a flow.


In [36]:
from jobflow import job

@job
def add(a, b, c=2):
    return a + b + c

@job
def mult(a, b):
    return a * b

To combine these two `Job` objects into a single workflow, we can take advantage of the `Flow` constructor in Jobflow.


In [37]:
from jobflow import Flow

job1 = add(1, 2)
job2 = mult(job1.output, 3)

flow = Flow([job1, job2], name="my-flow")

Because `job2` depends on the output of `job1`, it will only run if/when `job1` completes successfully. Jobflow will automatically determine the connectivity of the jobs and run them accordingly. Here, we have also given the flow an optional name, which can be useful for tracking purposes.

We can run the flow locally by calling the `run_locally` function.


In [38]:
from jobflow.managers.local import run_locally

responses = run_locally(flow)

2023-06-07 20:25:22,581 INFO Started executing jobs locally
2023-06-07 20:25:22,582 INFO Starting job - add (c7e3f989-b2a5-4c9d-8472-7cbc195970c6)
2023-06-07 20:25:22,584 INFO Finished job - add (c7e3f989-b2a5-4c9d-8472-7cbc195970c6)
2023-06-07 20:25:22,586 INFO Starting job - mult (e8266c45-d01a-40aa-8457-deda5603b9e1)
2023-06-07 20:25:22,587 INFO Finished job - mult (e8266c45-d01a-40aa-8457-deda5603b9e1)
2023-06-07 20:25:22,588 INFO Finished executing jobs locally


The output contains a UUID for each `job` in the flow along with the outputs for each job.


In [39]:
print(responses)

{'c7e3f989-b2a5-4c9d-8472-7cbc195970c6': {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}, 'e8266c45-d01a-40aa-8457-deda5603b9e1': {1: Response(output=15, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}}
